<img style="float: left;" src="earth-lab-logo-rgb.png" width="150" height="150" />

# Homework Template: Earth Analytics Python Course: Spring 2020

Before submitting this assignment, be sure to restart the kernel and run all cells. To do this, pull down the Kernel drop down at the top of this notebook. Then select **restart and run all**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below.

* IMPORTANT: Before you submit your notebook, restart the kernel and run all! Your first cell in the notebook should be `[1]` and all cells should run in order! You will lose points if your notebook does not run. 

For all plots and code in general:

* Add appropriate titles to your plot that clearly and concisely describe what the plot shows (e.g. time, location, phenomenon).
* Be sure to use the correct bands for each plot.
* Specify the source of the data for each plot using a plot caption created with `ax.text()`.
* Place ONLY the code needed to create a plot in the plot cells. Place additional processing code ABOVE that cell (in a separate code cell).

Make sure that you:

* **Only include the package imports, code, data, and outputs that are CRUCIAL to your homework assignment.**
* Follow PEP 8 standards. Use the `pep8` tool in Jupyter Notebook to ensure proper formatting (however, note that it does not catch everything!).
* Keep comments concise and strategic. Don't comment every line!
* Organize your code in a way that makes it easy to follow. 
* Write your code so that it can be run on any operating system. This means that:
   1. the data should be downloaded in the notebook to ensure it's reproducible.
   2. all paths should be created dynamically using the os package to ensure that they work across operating systems. 
* Check for spelling errors in your text and code comments


In [1]:
NAME = "Sarah Jaffe"
COLLABORATORS = "Ruby Shaheen"

![Colored Bar](colored-bar.png)

# Week 09 Homework - Multispectral Remote Sensing II


## Include the Plots, Text and Outputs Below

For all plots:

* Add appropriate titles to your plot that clearly and concisely describe what the plot shows.
* Be sure to use the correct bands for each plot.
* Specify the source of the data used for each plot in a plot caption using `ax.text()`.


## Project Introduction (10 points)

Read the overview of the cold springs fire: https://www.earthdatascience.org/courses/use-data-open-source-python/data-stories/cold-springs-wildfire/

In the Markdown cell below, add a 2-4 sentence description of the Cold Springs Fire. This should 
include the event:
1. name, 
2. type, 
3. duration / dates and 
4. location. 

Notifications of the Cold Springs Fire began on July 9th, 2016.  This fire on Hurricane Hill, two miles northeast of Nederland, was reported to have started by an improperly extinguished campfire, burned for 5 days (officially extinguished July 14th, 2016).  This fire resulted in the damage of 528 acres and 8 homes.

![Colored Bar](colored-bar.png)

In [2]:
# Autograding imports - do not modify this cell
import matplotcheck.notebook as nb
import matplotcheck.autograde as ag
import matplotcheck.raster as rs

In [3]:
# Import libraries (5 points) 
# Only include imports required to run this notebook
import os
from glob import glob
import warnings

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import re  

import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em

# Get Landsat data 
et.data.get_data(url="https://ndownloader.figshare.com/files/21941085")

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data')) 
warnings.simplefilter('ignore')

## Assignment Week 1 - Complete by March 18 

## Define Functions To Process Landsat Data

For next week (March 18), create the 3 functions below used to process landsat data.

For all functions, add a docstring that uses numpy style format (for examples, review the [Intro to Earth Data Sciene textbook chapter on Functions](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/functions-modular-code/write-functions-in-python/#docstring)). 

The docstring should include:

    * A one sentence description of what the function does.
    * Description of each input variable (parameter), following numpy docstring standards.
    * Description of each output object (return), following numpy docstring standards.

## Function 1: crop_stack_data function (5 points)

Write a function called `crop_stack_data` that: 
1. Takes a **list** of raster TIF files and crops all of the files in the list to a given input boundary in GeoPandas GeoDataFrame format.
    * **3 inputs:** 
        * 1) list of files (i.e. the files to crop).
        * 2) directory to export cropped files.
        * 3) GeoPandas GeoDataFrame to crop the data.
2. Returns a stacked **numpy array** of the cropped data and the metadata.
    * **2 outputs:**
        * 1) **numpy array**.
        * 2) **dictionary** containing metadata.

In [13]:
# Add your function here. Do NOT modify the function name
def crop_stack_data(files_to_crop, crop_dir_path, crop_bound):
    """Crops a set of tif files and saves them 
    in a crop directory. Returns a stacked numpy 
    array of bands.
    
    Parameters
    ----------
    files_to_crop : list
        List of paths to multispectrum scenes 
        (.tiff) that will need cropping.
    
    crop_dir_path : str
        The path to an output directory already 
        in existance, or will be made, and that 
        will store cropped and exported stacked 
        bands.
        
    crop_bound : GeoPandas GeoDataFrame
        Vector shape file geodataframe used for 
        cropping aoi's from files_to_crop.
    
    Returns
    -------
    all_bands_stack : numpy array(s)
        Stacked and cropped numpy array bands 
        (our new aoi's).
    
    fire_crop_utmz13 : GeoPandas GeoDataFrame
        A vector shape file that either shares 
        the crs of the stacked bands or is 
        reprojected from the crop_bound crs.
    """
    # Create output directory   
    if not os.path.exists(crop_dir_path):
        os.mkdir(crop_dir_path)

    # Reproject boundary .shp to geotiff crs
    with rio.open(files_to_crop[0]) as landsat_src:
        if not crop_bound.crs == landsat_src.crs:
            fire_crop_utmz13 = crop_bound.to_crs(
                                  landsat_src.crs)

    # Crop all geotiff layers in each scene
    es.crop_all(raster_paths=files_to_crop,
                output_dir=crop_dir_path,
                geoms=fire_crop_utmz13,
                overwrite=True)

    # Retrieve cropped bands from output directory
    all_bands = sorted(glob(os.path.join(
                       crop_dir_path, "*.tif")))
    
    # Stacked cropped bands
    all_bands_stack = es.stack(all_bands)

    
############################### DOES NOT INCLUDE DICTIONARY FOR META DATA #############################
    # Stacked cropped bands and new fire boundary
    return all_bands_stack, fire_crop_utmz13

## Function 2: mask_data (5 points)

In the call below, write a function called `mask_data` that: 
1. Masks a numpy array using the Landsat Cloud QA layer. 
    * **2 inputs:** 
        * 1) numpy array to be masked
        * 2) Landsat QA layer in numpy array format 
2. Returns a masked array. 
    * **1 output:**
        * 1) masked numpy array

In [14]:
# Add your function here. Do NOT modify the function name
def mask_data(arr, path_to_qa):
    """Function that masks a numpy array using a 
    cloud qa layer.
    
    Parameters
    ----------
    arr : numpy array
        Numpy array(s) of bands of aoi scene(s).        
    
    path_to_qa : str
        Path to QA layer(s) associated with aoi(s).        
    
    Returns
    -------
    arr : masked numpy array
        Updated numpy array(s) of bands with high cloud
        confidence, clouds and cloud shadows masked.        
    """
    # Open the qa layer
    with rio.open(path_to_qa[0]) as src:
        mask_arr = src.read(1)

    # Cloud mask values
    high_cloud_confidence = em.pixel_flags["pixel_qa"][
                                           "L8"][
                                           "High Cloud Confidence"]
    cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
    cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]

    all_masked_values = cloud_shadow + cloud + high_cloud_confidence
    
    # Mask the numpy array
    if any(i in np.unique(mask_arr) for i in all_masked_values):
        landsat_masked_bands = em.mask_pixels(arr,
                                              mask_arr,
                                              vals=all_masked_values)
        return landsat_masked_bands
    else:
        return arr

## Function 3: classify_dnbr (5 points)

In the cell below, write a function called `classify_dnbr` that: 
1. Classifies a numpy array using classes/bins defined in the function. 
    * **1 input:**
        * 1) numpy array containing dNBR data in numpy array format 
2. Returns a classified numpy array. 
    * **1 output:**
        * 1) numpy array with classified values (integers)

In [15]:
# Add your function here. Do NOT modify the function name
def classify_dnbr(arr):
    """Function that creates a new numpy array of classified
    values from a difference normalized burn ration (dNBR) 
    numpy array. 
    
    Parameters
    ----------
    arr : Numpy array
        Numpy array(s) containing dNBR data.        
    
    Returns
    -------
    arr_class : Numpy array
        Numpy array(s) containing reclassified 
        dNBR values in 5 possible classes.        
    """
    # Reclassify values #########NOTE: WK 2 NOTEBOOK##########
    class_bins = [-np.inf, -.1, .1, .27, .66, np.inf]
    arr_reclass = np.digitize(arr, class_bins)
        
    return arr_reclass   

## Assignment Week 2 - BEGINS March 18 

You will write the function below next week after learning more about MODIS h4 data in class on March 18, 2020.

Be sure to add a docstring that uses numpy style format (for examples, review the [Intro to Earth Data Sciene textbook chapter on Functions](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/functions-modular-code/write-functions-in-python/#docstring)). 

The docstring should include:

    * A one sentence description of what the function does.
    * Description of each input variable (parameter), following numpy docstring standards.
    * Description of each output object (return), following numpy docstring standards.
    
## Function 4: stack_modis_bands (10 points)

Write a function called `stack_modis_bands` that: 
1. Loops through an `h4` file to collect and stack all "band" layers in the file.
2. Crops each band to the extent of a given input boundary in GeoPandas GeoDataFrame format.
3. Returns a stacked numpy array of the cropped data and the metadata.

#### Function Inputs and Outputs

1. Takes a path to an h4 file and returns the reflectance bands cropped to a specific extent. 
    * **2 inputs:**
        * 1) string path to hdf h4 file
        * 2) crop_bound in GeoDataFrame format
2. Returns a classified numpy array. 
    * **2 outputs:**
        * 1) numpy array containing cropped MODIS reflectance bands
        * 2) metadata for the numpy array

In [16]:
# Add your function here. Do NOT modify the function name
def stack_modis_bands(h4_path, crop_bound):
    '''
    Accessing, cropping, stacking and cleaning (masking
    nodata) all bands within h4 files and producing
    outputs necessary to process and plot data.
    
    Parameters
    ----------
    h4_path : str
    Path(s) to h4 file(s).
    
    crop_bound : GeoPandas GeoDataFrame
        Vector shape file geodataframe used for 
        cropping aoi's from files_to_crop.
        
    Returns
    -------    
    cleaned_modis_data : numpy arrays
        Stacked and cropped numpy array bands 
        (our new aoi's).
        
    crop_meta : dict
        Dictionary containing details of 
        stacked numpy arrays in 
        cleaned_modis_data.
            
    extent_modis : tuple
        Boundary defined by crop_bound 
        required to plot aoi extent.
    
    fire_bound_sin : GeoPandas GeoDataFrame
        A vector shape file that either shares 
        the crs of the stacked bands or is 
        reprojected from the crop_bound crs. 
    '''
    # Temporary hold cropped, stacked bands
    processed_bands = []
    
    # Access bands and band data
    with rio.open(h4_path) as dataset:
        for name in dataset.subdatasets:
            if re.search("b0.\_1$", name): 
                with rio.open(name) as subdataset:

                    if not crop_bound.crs == subdataset.crs:
                        fire_bound_sin = crop_bound.to_crs(
                                                subdataset.crs)

                    # Crop bands read with fire boundary
                    crop_band, crop_meta = es.crop_image(
                                            subdataset,fire_bound_sin)
                    processed_bands.append(np.squeeze(crop_band))
                    
    # Stack
    modis_bands_stack = np.stack(processed_bands)                

    # Identify and clean array of nodata
    cleaned_modis_data = ma.masked_where(
        modis_bands_stack == crop_meta["nodata"], modis_bands_stack) 
    
    # Plotting boundary
    extent_modis = plotting_extent(
        crop_band[0], crop_meta["transform"])

    return cleaned_modis_data, crop_meta, fire_bound_sin, extent_modis


![Colored Bar](colored-bar.png)

# Figure 1 Overview - Grid of 3 Color InfraRed (CIR) Plots: NAIP, Landsat and MODIS

**You will be able to complete the MODIS subplot after March 18 class!**

Create a single figure that contains a grid of 3 plots of color infrared (also called false color) composite images using:

* Post Fire NAIP data (this is the data that you downloaded for your week 6 homework)
* Post Fire Landsat data (use: `et.data.get_data(url="https://ndownloader.figshare.com/files/21941085")`
* Post Fire MODIS data (use: `et.data.get_data('cold-springs-modis-h5')`

For each map, be sure to:

* Crop the data to the fire boundary extent.
* Overlay the fire boundary layer (`vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp`).
* Use the band combination **r = infrared band**, **g = red band**, **b = green** band.
* Be sure to label each plot with the data type (NAIP vs. Landsat vs. MODIS) and spatial resolution.

HINT: In a CIR image, the NIR band is plotted on the “red” band, the red band is plotted on the "green" band and the green band is plotted on the "blue" band.

In [19]:
# Open fire boundary in this cell
# Import fire boundary .shp
fire_path = os.path.join("cold-springs-fire",
                         "vector_layers",
                         "fire-boundary-geomac",
                         "co_cold_springs_20160711_2200_dd83.shp")
fire_crop = gpd.read_file(fire_path)
print(fire_crop.total_bounds)

# View data attributes and CRS.
print(fire_crop.shape)
print(fire_crop.crs)

[-105.49580578   39.97552258 -105.45633769   39.98718058]
(1, 22)
{'init': 'epsg:4269'}


## Process Landsat Data

In the cells below, open and process your Landsat Data using loops and string manipulation of paths. 

Use the functions `crop_stack_data()` and `mask_data()` that you wrote above to open, 
crop and stack each Landsat data scene (pre and post fire).

#### Notes

If you were implementing this workflow for more scenes, you could write 
a helper function that tested the crs of the crop extent. If it needed to be 
reprojected you could do so and write it out as a file, or store is in a dictionary
for for reuse 
in your loop. For this assignment rather than introducing additional tasks
we will keep it simple and open up and reproject the boundary once.

One way to do this is to create a connection to one single tif file using
rasterio. Once you have the `src` object you can grab the crs and reproject
the fire boundary. The code to do that is below:

```
with rio.open(glob(all_dirs[0] + "/*.tif")[0]) as src:
    fire_bound_utmz13 = fire_boundary.to_crs(src.crs)
```


In [20]:
# Create loop to process Landsat data in this cell
# Set paths to scene directories
base_path = os.path.join("earthpy-downloads", 
                         "landsat-coldsprings-hw")
                 
all_dirs = glob(os.path.join(base_path, "*"))

# Lists for naming convension dictionaries
cleaned_landsat_data = {}

# Set paths, create output and run functiond to crop/mask scenes 
for i in all_dirs:
    
    # Paths to all landsat scenes
    all_scenes = sorted(glob(os.path.join(i, "*.tif")))
    crop_path = os.path.join(i, "cropped")
    
    # Grabbing identifying names/dates
    scene_name = os.path.basename(os.path.normpath(i))
    date = scene_name[10:18]

    # Crop and stack all landsat scenes 
    stacked_bands, fire_crop_utmz13 = crop_stack_data(
                                      all_scenes, 
                                      crop_path, 
                                      fire_crop)
    
    # Paths to cropped qa and bands
    cropped_qa = glob(os.path.join(crop_path, 
                                  "*pixel**crop*.tif"))
    cropped_scenes = sorted(glob(os.path.join(crop_path, 
                                             "*band*")))
    
    # Mask all landsat scenes of bad pixels 
    bands_arr, bands_meta = es.stack(cropped_scenes, nodata=-9999)
    cleaned_landsat_data[date] = mask_data(bands_arr, cropped_qa)
    
    # Create plotting extent
    with rio.open(cropped_scenes[1]) as landsat_src:
        extent_landsat = plotting_extent(landsat_src)

IndexError: list index out of range

In [ ]:
cleaned_landsat_data['20160621']

In [ ]:
# Landsat NDVI processing
landsat_prefire_ndvi = es.normalized_diff(
    cleaned_landsat_data["20160621"][4], 
    cleaned_landsat_data["20160621"][3])
landsat_postfire_ndvi = es.normalized_diff(
    cleaned_landsat_data["20160723"][4], 
    cleaned_landsat_data["20160723"][3])
    
landsat_dndvi = landsat_postfire_ndvi - landsat_prefire_ndvi

## Landsat Function Tests

In [ ]:
# DO NOT MODIFY - test: crop_stack_data function

In [ ]:
# DO NOT MODIFY - test mask_data function


In [ ]:
# DO NOT MODIFY - test classify_dnbr function


## Process NAIP Post Fire Data

In the cell below, open and crop the post-fire NAIP data that you downloaded 
for homework 6.

In [ ]:
# Process NAIP data
#Import NAIP 2017 image
naip_2017_path = os.path.join("cold-springs-fire", "naip", 
                              "m_3910505_nw_13_1_20170902",
                              "m_3910505_nw_13_1_20170902.tif")

#######################NODATA NOT ACCOUNTED FOR HERE = "None"########################################
with rio.open(naip_2017_path) as naip_2017_src:
    if landsat_src.crs == naip_2017_src.crs:
        fire_crop_reproj = fire_crop_utmz13
    else:
        fire_crop_reproj = fire_crop.to_crs('epsg:26913')
    
    naip_2017_crop, naip_2017_meta = es.crop_image(
                                     naip_2017_src, fire_crop_reproj)
    naip_extent = plotting_extent(naip_2017_crop[0], 
                                  naip_2017_meta['transform'])

## Process MODIS h4 Data - March 18th, 2020

In the cells below, open and process your MODIS hdf4 data using loops and 
string manipulation of paths. You will learn more about working with MODIS 
in class on March 18th.

Use the function `stack_modis_bands()` that you previously wrote in this notebook to open and crop 
the MODIS data.


In [ ]:
# Process MODIS Data
# Set paths to scene directories
modis_dirs = glob(os.path.join("cold-springs-modis-h5", 
                               "*"))

# Modis dictionary of scene dates
modis_bands_dict = {}

for i in modis_dirs:
    
    # Paths to all modis scenes
    all_scenes = glob(os.path.join(i, "*.hdf"))
    
    # Grabbing identifying names/dates
    scene_names = os.path.basename(os.path.normpath(i))
    date = scene_names
    
    # Use modis function to stack and crop
    cleaned_modis_data, modis_meta, modis_boundary, extent_modis = stack_modis_bands(all_scenes[0], fire_crop)

    # Creating dictionary
    modis_bands_dict[date] = np.squeeze(cleaned_modis_data)

In [ ]:
# Process modis NDVI
modis_prefire_ndvi = es.normalized_diff(
    modis_bands_dict['07_july_2016'][1], 
    modis_bands_dict['07_july_2016'][0])
modis_postfire_ndvi = es.normalized_diff(
    modis_bands_dict['17_july_2016'][1], 
    modis_bands_dict['17_july_2016'][0])
    
modis_dndvi = modis_postfire_ndvi - modis_prefire_ndvi

In [ ]:
# DO NOT MODIFY THIS CELL - autograding tests for MODIS function stack_modis_bands


## Figure 1: Plot CIR for NAIP, Landsat and MODIS Using Post Fire Data (15 points each subplot)

In the cell below, create a figure with 3 subplots stacked vertically.

In each subplot, plot a CIR composite image using the post-fire data for:

* NAIP  (first figure axis) 
* Landsat (second figure axis) 
* MODIS (third figure axis)

respectively on this figure.

In [1]:
# Plot CIR of Post Fire NAIP, Landsat & MODIS together in one figure
fig, [ax1, ax2, ax3] = plt.subplots(3, 1, figsize=(12, 18))

# Plot NAIP CIR
ep.plot_rgb(naip_2017_crop,
            extent = naip_extent,
            rgb = [3, 0, 1],
            ax = ax1,
            title="NAIP CIR Image\n Post Cold Springs" \
                  " Fire, Colorado\n 2 September 2017")

fire_crop_reproj.plot(ax = ax1, color = 'None', 
                           edgecolor = 'white', linewidth=2)

# Plot Landsat CIR
ep.plot_rgb(cleaned_landsat_data["20160723"],
            rgb=[4,3,2],
            extent=extent_landsat,
            ax=ax2,
            title = "Landsat CIR Composit Image\n Post Cold Springs" \
                    " Fire, Colorado\n 23 July 2016")

fire_crop_utmz13.plot(ax=ax2, color="None", 
                      edgecolor="white", linewidth=2)

# Plot modis CIR
ep.plot_rgb(modis_bands_dict['17_july_2016'],
            rgb=[4,3,2],
            extent=extent_modis,
            ax=ax3,
            title = "Modis CIR Composit Image\n Post Cold Springs" \
                    " Fire, Colorado\n 17 July 2016")

modis_boundary.plot(ax=ax3, color="None", 
                    edgecolor="white", linewidth=2)


# Captions
ax1.text(0, -0.1,'2017 1m NAIP Image Data Source: Earth Explorer\n'
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom', 
         horizontalalignment='left', transform=ax1.transAxes)

ax2.text(0, -0.1, '2017 30m Landsat Image Data Source: ' \
         'https://ndownloader.figshare.com/files/21941085\n' 
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom',  
         horizontalalignment='left', transform=ax2.transAxes)

ax3.text(0, -0.1, '2017 500m MODIS Image Data Source: earthpy ' \
         'module key "cold-springs-modis-h5" \n'
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom',  
         horizontalalignment='left', transform=ax3.transAxes)

### DO NOT REMOVE LINE BELOW ###
plot01_CIR_res_comparison = nb.convert_axes(plt, which_axes="all")

NameError: name 'plt' is not defined

In [ ]:
# DO NOT TOUCH THIS CELL - autograding tests for NAIP subplot


In [ ]:
# DO NOT TOUCH THIS CELL - autograding tests for Landsat subplot


In [ ]:
# DO NOT TOUCH THIS CELL - autograding tests for MODIS subplot


![Colored Bar](colored-bar.png)

# Figure 2: Difference NDVI (dNDVI) Using Landsat & MODIS Data (20 points each subplot)

Plot the NDVI difference using before and after Landsat and MODIS data 
that cover the Cold Springs Fire study area. For each axis be sure to:

1. overlay the fire boundary (`vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp`) on top of the data.
2. Be sure that the data are cropped using the fire boundary extent.

In the cell below, create a figure with 2 subplots stacked vertically:
* Plot dNDVI for Landsat on the first axis of the figure.
* Plot dNDVI using MODIS on the second axis of the figure.

Use the "before" and "after" data that you processed above to calculate NDVI difference for both MODIS and Landsat


## NDVI Difference

To create the NDVI Difference raster using "before" and "after" fire 
Landsat and MODIS data, you must first calculate NDVI for each 
dataset "before" and "after" the fire. 

Once you have the "before" and "after" NDVI arrays, you can subtract 
the pre-fire NDVI array FROM the post-fire NDVI array (post-fire minus pre-fire). 

The resulting array will show you change in the area's NDVI from the first image to the second image.

HINT: Remember, you can use `es.normalized_diff(band_1, band_2)` to get the NDVI of an image. 

In [ ]:
# Plot Difference NDVI for Landsat & MODIS together in one figure
fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(12, 12))

# Plot Landsat dNDVI
ep.plot_bands(landsat_dndvi, cmap="RdYlGn",
              vmin=-0.6, vmax=0.6, ax = ax1, extent=extent_landsat,
              title="Landsat Derived dNDVI\n 21 June vs. 23 July," \
                    " 2016\n Cold Springs Fire, Colorado", scale=False)

fire_crop_utmz13.plot(ax=ax1, color='None', 
                      edgecolor='black', linewidth=2)

# Plot modis dNDVI
ep.plot_bands(modis_dndvi, cmap="RdYlGn",
              vmin=-0.6, vmax=0.6, ax = ax2, extent=extent_modis,
              title="Modis Derived dNDVI\n 07 July vs. 17 July," \
                    " 2016\n Cold Springs Fire, Colorado", scale=False)

modis_boundary.plot(ax=ax2, color='None', 
                    edgecolor='black', linewidth=2)

# Captions
ax1.text(0, -0.1, '2017 30m Landsat Image Data Source: ' \
         'https://ndownloader.figshare.com/files/21941085\n' 
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom',  
         horizontalalignment='left', transform=ax1.transAxes)

ax2.text(0, -0.1, '2017 500m MODIS Image Data Source: earthpy ' \
         'module key "cold-springs-modis-h5" \n'
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom',  
         horizontalalignment='left', transform=ax2.transAxes)

### DO NOT REMOVE LINE BELOW ###
plot02_landsat_modis_ndvi_diff = nb.convert_axes(plt, which_axes="all")

In [ ]:
# Ignore this cell - autograding tests for Landsat subplot


In [ ]:
# Ignore this cell - autograding tests for MODIS subplot


![Colored Bar](colored-bar.png)

# Figure 3 Overview: Difference NBR (dNBR) Using Landsat  & MODIS Data (25 points each subplot)

Create a figure that has two subplots stacked vertically using the same MODIS and Landsat data that you processed above. 

* Subplot one: classified dNBR using Landsat data
* Subplot two: classified dNBR using MODIS data 

For each subplot, overlay the fire extent boundary `vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp`
on top of the dNBR map

To classify each dNBR raster, use the `classify_dnbr()` function that you 
defined above. 

When you plot your MODIS data, you may notice that the data does not contain all of the classes that Landsat contains which can range from 1-5. To ensure that your colormap plots properly, set the `vmin=` and `vmax=` parameters to 1 and 5 respectively when you call `ep.plot_bands()`:

`vmin=1, vmax=5`


## Figure Legend

You only need one legend for this figure. The `ep.draw_legend()` function will create a legend of "boxes" if you provide it with an:

1. `imshow()` image object
2. classes : a list of numbers that represent the classes in your numpy array
3. titles: a list of dNBR class names example: `["High Severity", "Low Severity"]`

### dNBR Classes

Note: if you scaled your data, you may need to scale the values below by a factor of 10.

| SEVERITY LEVEL  | dNBR RANGE |
|----------------|--------------|
| Enhanced Regrowth | < -.1 |
| Unburned        | -.1 to +.1 |
| Low Severity     | +.1 to +.27 |
| Moderate Severity  | +.27 to +.66 |
| High Severity    | > .66 |


HINT: Your dNBR classification list should look like this:
`[-np.inf, -.1, .1, .27, .66, np.inf]`

HINT 2: If you want to use them, these are the colors used in the maps on the website:

`["g", "yellowgreen", "peachpuff", "coral", "maroon"]`

The code to create a custom colormap for the plot is below:

```
nbr_colors = ["g", "yellowgreen", "peachpuff", "coral", "maroon"]
nbr_cmap = ListedColormap(nbr_colors)
```

In [ ]:
# Calculate dNBR for Landsat - be sure to use the correct bands!
# Landsat NBR processing
landsat_prefire_nbr = es.normalized_diff(
    cleaned_landsat_data["20160621"][4], 
    cleaned_landsat_data["20160621"][6])
landsat_postfire_nbr = es.normalized_diff(
    cleaned_landsat_data["20160723"][4], 
    cleaned_landsat_data["20160723"][6])
    
landsat_dnbr = landsat_prefire_nbr - landsat_postfire_nbr

landsat_dnbr_reclass = classify_dnbr(landsat_dnbr)

In [ ]:
# Calculate dNBR for MODIS - be sure to use the correct bands!
# Modis NBR Processing
modis_prefire_nbr = es.normalized_diff(
    modis_bands_dict['07_july_2016'][1], 
    modis_bands_dict['07_july_2016'][6])
modis_postfire_nbr = es.normalized_diff(
    modis_bands_dict['17_july_2016'][1], 
    modis_bands_dict['17_july_2016'][6])
    
modis_dnbr = modis_prefire_nbr - modis_postfire_nbr

modis_dnbr_reclass = classify_dnbr(modis_dnbr)

In [ ]:
# Plot Difference NBR (dNBR) for Landsat & MODIS together in one figure
fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(12, 12))

colors = ["g", "yellowgreen", "peachpuff", "coral", "maroon"]
class_labels = ["Enhanced Regrowth", "Unburned", "Low Severity",
               "Moderate Severity", "High Severity"]
cmap = ListedColormap(colors)

# Plot Landsat dNBR
im = ax1.imshow(landsat_dnbr_reclass, cmap = cmap, 
                extent=extent_landsat)

fire_crop_utmz13.plot(ax=ax1, color='None', 
                      edgecolor='black', linewidth=2)

ep.draw_legend(im, titles=class_labels)
ax1.set_title("Landsat Derived dNBR\n 21 June vs 23 July " \
          "2016\n Cold Springs Fire, Colorado")
ax1.set_axis_off()

# Plot Modis dNBR
im2 = ax2.imshow(modis_dnbr_reclass, cmap = cmap, 
                vmin=1, vmax=5, extent=extent_modis)

modis_boundary.plot(ax=ax2, color='None', 
                    edgecolor='black', linewidth=2)

ax2.set_title("Modis Derived dNBR\n 21 June vs 23 July " \
              "2016\n Cold Springs Fire, Colorado")
ax2.set_axis_off()

# Captions
ax1.text(0, -0.1, '2017 30m Landsat Image Data Source: ' \
         'https://ndownloader.figshare.com/files/21941085\n' 
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom',  
         horizontalalignment='left', transform=ax1.transAxes)

ax2.text(0, -0.1, '2017 500m MODIS Image Data Source: earthpy ' \
         'module key "cold-springs-modis-h5" \n'
         r'Boundary Data Source: Geospatial Multi-Agency ' \
         r'Coordination (GeoMAC)', verticalalignment='bottom',  
         horizontalalignment='left', transform=ax2.transAxes)

### DO NOT REMOVE LINE BELOW ###
plot03_landsat_dnbr = nb.convert_axes(plt, which_axes="all")

In [ ]:
# Ignore this cell - autograding tests for Landsat subplot


In [ ]:
# Ignore this cell - autograding tests for MODIS subplot


![Colored Bar](colored-bar.png)

# Landsat vs MODIS  Burned Area (10 points)

In the cell below, print the total area burned in classes 4 and 5 (moderate to high severity) for both datasets 
(Landsat and MODIS).

HINT: Feel free to experiment with loops to complete this part of the homework. 

In [ ]:
# Total Burned Area in Classes 4 and 5 for Landsat and MODIS
landsat_pixel_area = int(landsat_src.res[0]) * int(landsat_src.res[0])
modis_pixel_area = int(modis_meta['transform'][0]) * \
    int(modis_meta['transform'][0])

burned_area_per_source = [landsat_dnbr_reclass, modis_dnbr_reclass]
burned_area_per_pixel = [landsat_pixel_area, modis_pixel_area]
total_burned_area = []

# for i, z in zip(burned_area_per_source, burned_area_per_pixel):
#     area_loop = ((((i[i == 4]).size)*z) + (((i[i == 5]).size)*z))
#     total_burned_area.append(area_loop)

print('Landsat total burned area (meters squared):', 
      total_burned_area[0])
print('Modis total burned area (meters squared):', 
      total_burned_area[1])

landsat_class_4 = (landsat_dnbr_reclass[landsat_dnbr_reclass == 4]).size
landsat_class_5 = (landsat_dnbr_reclass[landsat_dnbr_reclass == 5]).size

modis_class_4 = (modis_dnbr_reclass[modis_dnbr_reclass == 4]).size
modis_class_5 = (modis_dnbr_reclass[modis_dnbr_reclass == 5]).size

landsat_mod_severity_area = landsat_pixel_area * landsat_class_4
landsat_high_severity_area = landsat_pixel_area * landsat_class_5

landsat_total_burn_area = landsat_mod_severity_area + landsat_high_severity_area

modis_mod_severity_area = modis_pixel_area * modis_class_4
modis_high_severity_area = modis_pixel_area * modis_class_5

modis_total_burn_area = modis_mod_severity_area + modis_high_severity_area

print('Landsat total burned area:', landsat_total_burn_area, 'meters')
print('Modis total burned area:', modis_total_burn_area, 'meters')

![Colored Bar](colored-bar.png)

# Do not edit this cell! (5 points)

* Each figure specifies the source of the data (for each plot) using a plot caption created with `ax.text()`.

# Do not edit this cell! (5 points)

The notebook will also be checked for overall clean code requirements as specified at the **very top** of this notebook! Some of these requirements include (review the top cells for more specifics): 

* Notebook begins at cell [1] and runs on any machine in its entirety.
* PEP 8 format is applied throughout (including lengths of comment and code lines).
* No additional code or imports in the notebook
* Notebook is fully reproducible. This means:
   * reproducible paths using the os module.
   * data downloaded using code in the notebook.
   * all imports at top of notebook.

# Do not edit this cell!  (5 points)

All functions contain docstrings with inputs and outputs clearly identified and following numpy docstring standards.